<a href="https://colab.research.google.com/github/VMBoehm/N3AS_Project_Malika/blob/main/NeutrinoClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
#! ls ./drive/MyDrive/

In [58]:
class MassiceNusGen(tf.keras.utils.Sequence):
    
    def __init__(self, path, redshift, batch_size, file_num=12768, size=(128, 128,1), shuffle=True, seed=9456, valid=False):
        
        self.path       = path
        self.z          = redshift
        self.batch_size = batch_size
        self.dim        = size
        self.shuffle    = shuffle
        self.file_num   = file_num
        self.valid_size = self.file_num//5
        self.train_size = self.file_num-self.valid_size   
        self.size       = size
        self.folder     = "z%s"%str(int(10*self.z)).zfill(2)
        
        if valid:
            self.order      = np.arange(self.train_size,self.file_num)
            self.size       = self.valid_size
        else:
            self.order      = np.arange(self.train_size)
            self.size       = self.train_size
        
        np.random.seed(seed)
        np.random.shuffle(self.order)
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.order)
    
    def __get_map(self, data):
        data_array = tf.convert_to_tensor(data, dtype=tf.float32)  
        data_array = tf.expand_dims(data_array,-1)
        data_array = tf.image.resize(data_array,self.dim[0:2])
        
        
        return data_array
    
    def __load_label(self, name):
        hdul = fits.open(os.path.join(name))
        mnu  = hdul[0].header["MNU"]
        label= int(mnu>0)
        return label

    def __get_name(self, ind):
        file   = "%d.fits"%ind
        return os.path.join(self.path,self.folder,file)
    
    def __load_data(self,name):
        hdul = fits.open(os.path.join(name))
        data = self.__get_map(hdul[1].data)
        return data
        
    def __get_data(self, inds):
        X  = np.asarray([self.__load_data(self.__get_name(ind)) for ind in inds])
        Y  = np.asarray([self.__load_label(self.__get_name(ind)) for ind in inds], dtype=np.int32)
        return X, Y
    
    def __getitem__(self, index):
        inds   = self.order[index*self.batch_size:(index+1)*self.batch_size]
        X,Y    = self.__get_data(inds)
        return X, Y
    
    def __len__(self):
        return self.size//self.batch_size


In [59]:
train_gen = MassiceNusGen(path='./drive/MyDrive/', redshift=1.5, batch_size=16, valid=False, size=(64, 64,1))

valid_gen = MassiceNusGen(path='./drive/MyDrive/', redshift=1.5, batch_size=16, valid=True, size=(64, 64,1))

In [60]:
# base_model = keras.applications.Xception(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(128, 128, 3),
#     include_top=False) 

base_model = tf.keras.applications.VGG16(weights='imagenet',  # Load weights pre-trained on ImageNet.
input_shape=(64, 64, 3),
include_top=False) 

In [61]:
X = next(iter(train_gen))

In [62]:
X

(array([[[[-1.60252005e-02],
          [-2.48812023e-03],
          [ 1.58180576e-03],
          ...,
          [-4.57319850e-03],
          [-1.07638203e-02],
          [-7.74525711e-03]],
 
         [[-2.31384486e-02],
          [-9.26744938e-03],
          [ 5.08536724e-03],
          ...,
          [ 8.24719109e-03],
          [-9.93038900e-03],
          [-1.53473802e-02]],
 
         [[-1.56499818e-02],
          [ 5.51103614e-04],
          [ 4.92653623e-03],
          ...,
          [ 2.08623149e-02],
          [ 2.99129598e-02],
          [ 9.45911929e-03]],
 
         ...,
 
         [[-8.38653743e-03],
          [-3.17078643e-03],
          [-2.32977746e-03],
          ...,
          [ 2.45546140e-02],
          [ 6.14186637e-02],
          [ 5.33008501e-02]],
 
         [[ 3.87175404e-03],
          [ 7.51744024e-03],
          [ 4.13263030e-03],
          ...,
          [ 1.77662987e-02],
          [ 3.95247117e-02],
          [ 5.78929037e-02]],
 
         [[ 3.36873010e-

In [63]:
max = np.max(X[0].flatten())
print(max)

0.35358882


In [71]:
base_model.trainable = False

In [65]:
max = 0.5 


In [66]:
from keras.layers import *
from keras import backend as K
from keras.models import Model
import numpy as np 

class Gray2VGGInput( Layer ) :
    """Custom conversion layer
    """
    def build( self, x ) :
        self.built = True
        return

    def call( self, x ) :
        rgb_x = K.concatenate( [x,x,x], axis=-1 )
        norm_x = rgb_x
        return norm_x

    def compute_output_shape( self, input_shape ) :
        return input_shape[:3] + (3,)

In [67]:


inputs = keras.Input(shape=(64, 64, 1))
vgg_input_image = Gray2VGGInput( name='gray_to_rgb_norm')(inputs)
x = vgg_input_image
scale_layer = keras.layers.Rescaling(scale=1 / max, offset=0)
x = scale_layer(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(8, activation='relu')(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [68]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [72]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 64, 64, 1)]       0         
                                                                 
 gray_to_rgb_norm (Gray2VGGI  (None, 64, 64, 3)        0         
 nput)                                                           
                                                                 
 rescaling_8 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 global_average_pooling2d_8   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 8)                 4104

In [73]:
# TASK: add validation data (https://keras.io/api/models/model_training_apis/)
model.fit(train_gen, epochs=1, validation_data=valid_gen)

 28/638 [>.............................] - ETA: 40:48 - loss: 0.6902 - binary_accuracy: 0.5915

KeyboardInterrupt: ignored